# Single Train Gradient Update

## Importing Libraries

In [1]:
from typing import Dict
import threading

import gym
import numpy as np
import torch as th

from stable_baselines3 import A2C as ALGO
from stable_baselines3.common.evaluation import evaluate_policy

## Init. ENV and Model

In [2]:
env = gym.make('LunarLander-v2')
model = ALGO(
    "MlpPolicy",
    env
)

## Functions to Evaluate Model and Train Model within Thread

In [3]:
def evaluate(model, env, message = '', verbose = 0):
    fitnesses = []
    iterations = 5
    for i in range(iterations):
        fitness, _ = evaluate_policy(model, env)
        if verbose == 1:
            print(i, fitness, end=" ")
        fitnesses.append(fitness)

    mean_fitness = np.mean(sorted(fitnesses))
    print(f'Type {message} Mean reward: {mean_fitness}')

In [4]:
def train(model, timesteps):
    # print('Starting Training')
    model.learn(total_timesteps=timesteps)
    # print('Completed Training')

## Initial Evaluation

In [5]:
evaluate(model, env)

/home/arnab/.miniconda3/envs/pydrl/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Type  Mean reward: -512.9018417249993


## Train for 1K Steps and Evaluate

In [6]:
# Train MT Model 1
t1 = threading.Thread(target=train, args=(model, 100_00))

# starting thread
t1.start()

# wait until thread is completely executed
t1.join()


# model_trained.learn(total_timesteps=10_00)
evaluate(model, env)

Type  Mean reward: -1928.619374931915


## Apply Gradient and Evaluate

In [7]:
for i in range(30):
    print('Train Iter: ', i, end="| ")

    # Train MT Model 1
    t1 = threading.Thread(target=train, args=(model, 10_000))

    # starting thread
    t1.start()
    
    # wait until thread is completely executed
    t1.join()
    
    evaluate(model, env, 'Updated Model', verbose=0)

Train Iter:  0| Type Updated Model Mean reward: -36.04365575263608
Train Iter:  1| Type Updated Model Mean reward: -13.336277716994028
Train Iter:  2| Type Updated Model Mean reward: 21.496134894605074
Train Iter:  3| Type Updated Model Mean reward: 67.22252640569847
Train Iter:  4| Type Updated Model Mean reward: 5.0157438332324515
Train Iter:  5| Type Updated Model Mean reward: -1.1861431246021794
Train Iter:  6| Type Updated Model Mean reward: 14.744326067112942
Train Iter:  7| Type Updated Model Mean reward: -26.196131752531176
Train Iter:  8| Type Updated Model Mean reward: 33.57936811245884
Train Iter:  9| Type Updated Model Mean reward: -59.65191638753985
Train Iter:  10| Type Updated Model Mean reward: -80.74126358374453
Train Iter:  11| Type Updated Model Mean reward: -79.85805602579156
Train Iter:  12| Type Updated Model Mean reward: -121.78600250537681
Train Iter:  13| Type Updated Model Mean reward: -109.42889896457521
Train Iter:  14| Type Updated Model Mean reward: -102.9

In [8]:
model.get_parameters()

{'policy': OrderedDict([('mlp_extractor.policy_net.0.weight',
               tensor([[ 1.5475e-01, -2.6161e-01,  2.6454e-01, -1.0527e+00, -1.5317e+00,
                        -2.3494e-01,  3.7251e-02,  1.3113e-01],
                       [-1.7597e-02,  1.8325e-01, -3.1351e-01, -1.1294e+00,  1.5014e+00,
                         3.3917e-01, -1.2012e-01, -2.5557e-01],
                       [-1.0264e-01, -1.0549e-01,  2.9058e-01, -1.5490e+00, -2.4721e-01,
                        -1.5816e-01, -7.8698e-02, -1.2077e-01],
                       [ 1.0099e-01, -1.4479e-01,  4.3260e-01, -4.5507e-01, -1.6883e+00,
                        -3.0802e-01, -9.1658e-02,  1.4004e-01],
                       [-1.2442e-01,  9.0441e-02,  5.6980e-02,  1.2400e+00, -1.8264e+00,
                        -3.2775e-01, -1.1673e-01,  1.8779e-01],
                       [ 1.8012e-01, -2.7934e-02,  1.3113e-01, -1.1815e+00, -1.1319e+00,
                        -2.5464e-01, -2.8528e-01,  1.4557e-01],
                    

In [9]:
model.save('a2c_lunar_singleproc')

In [10]:
# Exporting Params as JSON
## Function to Convert Params Dict to Flattened List
def flatten_list(params):
    """
    :param params: (dict)
    :return: (np.ndarray)
    """
    params_ = {}
    for key in params.keys():
        params_[key] = params[key].tolist()
    return params_
## Write Parameters to JSON File
import json

all_params = model.get_parameters()
pol_params = flatten_list(all_params['policy'])

all_params['policy'] = pol_params

with open('a2c_lunar_singleproc.json', 'w') as f:
    json.dump(all_params, f, indent='\t')

TypeError: Object of type Tensor is not JSON serializable

In [ ]:
model_loaded = ALGO(
    "MlpPolicy",
    env
)

evaluate(model_loaded,env, verbose=1)

new_params = all_params
loaded_pol_params = new_params['policy']
for key in loaded_pol_params.keys():
    loaded_pol_params[key] = th.tensor(loaded_pol_params[key])

new_params['policy'] = loaded_pol_params

model_loaded.set_parameters(new_params)

In [ ]:
env.reset()
evaluate(model_loaded,env, verbose=1)